[Back to Index](index.ipynb)

# Get Data for a Location

Begin by importing the required python modules.

In [ ]:
import requests
import pprint # useful for printing pretty json

Now, let's get the Raster Series Instance of Annual Average Maximum Temperature for one model (CNRM-CM5) and one scenario (RCP 4.5) from the Cal-Adapt API. Let's look at the `rasters` property.

In [ ]:
response = requests.get('http://api.cal-adapt.org/api/series/tasmax_year_CNRM-CM5_rcp45/')
response.json()

To get a list of all Raster Stores in a Raster Series Instance, append `rasters/` to the url.

In [ ]:
response = requests.get('http://api.cal-adapt.org/api/series/tasmax_year_CNRM-CM5_rcp45/rasters/')
response.json()

There are a total of 95 Raster Stores in this Raster Series Instance. By default only 10 are returned. You can use the `pagesize` query parameter to get more records with one request.

In [ ]:
response = requests.get('http://api.cal-adapt.org/api/series/tasmax_year_CNRM-CM5_rcp45/rasters/?pagesize=95')

# It is a good idea to check there were no problems with the request.
if response.status_code == requests.codes.ok:
    data = response.json()
    # Get a list of Raster Stores from results property of data object
    results = data['results']
    # Print the first item in list
    pprint.pprint(results[0])
    # Print a blank line
    print()
    # Iterate through the list and print the event property of each Raster Store
    for item in results:
        print(item['event'])

### Your turn

1. In the `results` loop in above cell print other properties of `item`. E.g.:
    - `print(item['image'])`

### Introducing the `g` parameter

By default, `image` field in a Raster Store Instance points to the entire geotiff file, e.g. `'http://api.cal-adapt.org/media/img/tasmax_year_CNRM-CM5_rcp45_r1i1p1_2006.LOCA_2016-04-02.16th.CA_NV.tif`. To get the data for a specific location we need to give the API a geometry object representing that location. The API can read geometry in several different formats inclduing GeoJSON, WKT. For this exercise we will use GeoJSON.

You also need to tell the API what format you want your data in. You can specify format by adding a `format` parameter to the query, e.g. `http://api.cal-adapt.org/api/series/tasmax_year_CNRM-CM5_rcp45/rasters/?pagesize=95&format=json`. Another way to do it is by sending a [HTTP header](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers) with the request, which is what we do below.

In [ ]:
# Query parameters dict
params = {
    'pagesize': 100,
    'g': '{"type":"Point","coordinates":[-121.4687,38.5938]}'
}

# Convert query parameters dict to string
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())

# Structure request url
url = 'http://api.cal-adapt.org/api/series/tasmax_year_CNRM-CM5_rcp45/rasters/?' + params_str

# HTTP header
headers = {'ContentType': 'json'}

# Make request
response = requests.get(url, headers=headers)

# It is a good idea to check there were no problems with the request.
if response.status_code == requests.codes.ok:
    data = response.json()
    # Get a list of Raster Stores
    results = data['results']
    print('First Raster Store object:')
    pprint.pprint(results[0])
    print()
    print('Timeseries for the grid cell at this point:')
    # Iterate through the list and print the event and image property of each Raster Store
    for item in results:
        print('year:', item['event'], 'value:', item['image'], item['units'])

### Your turn

1. Try another location by changing the coordinate pair in `params.g` in the above cell. The spatial extent available on Cal-Adapt covers entire state of California and Nevada and parts of Oregon, Mexico and Arizona.


### Get data for an area

You can also pass in polygon geometry. A polygon can span multiple grid cells (the grid is ~ 6km x 6km). With polygon geometry you can also provide a `stat` parameter which will aggregate data from all grid cells intersected by the polygon. The `stat` parameter can take the value `mean`, `max`, `min`.  

In [ ]:
g = '{"type":"Polygon","coordinates":[[[-122.30598,37.76881],[-122.22942,37.76881],[-122.229423,37.83337],[-122.305984,37.83337],[-122.305984,37.76881]]]}'
params = {
    'pagesize': 100,
    'g': g,
    'stat': 'mean'
}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
url = 'http://api.cal-adapt.org/api/series/tasmax_year_CNRM-CM5_rcp45/rasters/?' + params_str

# HTTP header
headers = {'ContentType': 'json'}

# Make request
response = requests.get(url, headers=headers)

# It is a good idea to check there were no problems with the request.
if response.status_code == requests.codes.ok:
    data = response.json()
    # Get a list of Raster Stores
    results = data['results']
    # Iterate through the list and print the event and image property of each Raster Store
    for item in results:
        print('year:', item['event'], 'value:', item['image'], item['units'])

### Use vector data from Cal-Adapt API

The API contains several boundary datasets. You can use polygon geometry from these vector resources instead of providing your own. For an example see notebook - [get_polygon_geometry_from_api](get_polygon_geometry_from_api.ipynb)

### Get data for a time slice

You can get data for a time slice instead of the full timeseries by substituting `rasters/` with a `begin-date/end-date/`. 

In [ ]:
g = '{"type":"Polygon","coordinates":[[[-122.30598,37.76881],[-122.22942,37.76881],[-122.229423,37.83337],[-122.305984,37.83337],[-122.305984,37.76881]]]}'
params = {
    'pagesize': 100,
    'g': g,
    'stat': 'mean'
}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
url = 'http://api.cal-adapt.org/api/series/tasmax_year_CNRM-CM5_rcp45/2070-01-01/2099-01-01/?' + params_str

# Add HTTP header
headers = {'ContentType': 'json'}

# Make request
response = requests.get(url, headers=headers)

# It is a good idea to check there were no problems with the request.
if response.status_code == requests.codes.ok:
    data = response.json()
    # Get a list of Raster Stores from results property of data object
    results = data['results']
    # Iterate through the list and print the event and image property of each Raster Store
    for item in results:
        print('year:', item['event'], 'value:', item['image'], item['units'])